In [3]:

import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline as imPipeline
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from spacy.lang.en.stop_words import STOP_WORDS


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
# For Google colab only

df = pd.read_csv("../kindle_reviews_2million.csv")
df = df[df['reviewText'].notna()]
df = df[~df['reviewText'].str.contains(".jpg|.png|.jpeg|.tiff|.gif|.bmp|.heif", regex=True, na=False)]

Split data

In [2]:
df = df[df['overall'] != 3]
df['overall'] = np.where(df['overall'] > 3, 1, 0)


NameError: name 'df' is not defined

In [5]:
X = df.reviewText.values
y = df.overall.values


In [6]:
num = len(df)
X, y  = X[:num], y[:num]
le = LabelEncoder()
y = le.fit_transform(y)

In [41]:
# vectoriser = TfidfVectorizer(analyzer='word', max_features=5000)
# X = vectoriser.fit_transform(X)
# rus = RandomUnderSampler()
# X_rus, y_rus = rus.fit_sample(X, y)

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:

def train_predict(pipline):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    score = classification_report(y_test, y_pred)
    #score = roc_auc_score(y_test, y_pred, average="macro")
    #print('AUC: ', roc_auc_score(y_test, y_pred))
    print(score)


Vectorizers


In [9]:
count_vectoriser = Pipeline([
                ('countVectoriser', CountVectorizer())
            ])

tfidf_vectoriser = Pipeline([
                ('tfidfVectoriser', TfidfVectorizer(stop_words=STOP_WORDS
                                                    ))
            ])

#ngram_range=(2,3), max_features=5000


In [19]:
def make_balance_pipe(steps):
    
    
    x = list(range(len(steps)))
    pipeline= [(str(x),y) for x,y in zip(x,steps)]

    return imPipeline(pipeline)


Naive Bayes


In [17]:
naive_bayes = Pipeline([
    ('classifier', MultinomialNB()),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', naive_bayes)
])

# pipeline = make_balance_pipe([TfidfVectorizer(),
#                               RandomUnderSampler(),
#                               MultinomialNB()])


In [20]:
train_predict(pipeline)

              precision    recall  f1-score   support

           0       0.58      0.58      0.58    163059
           1       0.42      0.46      0.44    163100
           2       0.46      0.39      0.42    162996
           3       0.47      0.43      0.45    161668
           4       0.63      0.72      0.67    161597

    accuracy                           0.51    812420
   macro avg       0.51      0.52      0.51    812420
weighted avg       0.51      0.51      0.51    812420



Max Entropy

In [10]:
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [11]:

max_ent = Pipeline([
    ('classifier', LogisticRegression(penalty='l2', C=1.0)),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', max_ent)
])

In [12]:
train_predict(pipeline)

/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


              precision    recall  f1-score   support

           0       0.60      0.62      0.61    163059
           1       0.45      0.46      0.45    163100
           2       0.47      0.42      0.44    162996
           3       0.50      0.49      0.49    161668
           4       0.67      0.71      0.69    161597

    accuracy                           0.54    812420
   macro avg       0.54      0.54      0.54    812420
weighted avg       0.54      0.54      0.54    812420



Logistic Regression

In [32]:
params={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
logistic_regression = Pipeline([
    ('classifier', GridSearchCV(LogisticRegression(), params, cv=10)),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', logistic_regression)
])

In [33]:
train_predict(pipeline)




/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 p

AUC:  0.5165617594079525
              precision    recall  f1-score   support

           0       0.52      0.48      0.50     13232
           1       0.51      0.55      0.53     13168

    accuracy                           0.52     26400
   macro avg       0.52      0.52      0.52     26400
weighted avg       0.52      0.52      0.52     26400



### Bagging Models

Random Forest

In [21]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [22]:
# random_forest = Pipeline([
#     ('classifier', RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = random_grid,
#                                       n_iter = 100, cv = 3, verbose=2, random_state=42,
#                                       n_jobs = -1)),
# ])

random_forest = Pipeline([
    ('classifier', RandomForestClassifier()),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', random_forest)
])

In [ ]:
train_predict(pipeline)

### Boosting Models

XGBoost


In [1]:
# !wget https://s3-us-west-2.amazonaws.com/xgboost-wheels/xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl
# !pip uninstall xgboost --yes
# !pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

# For Google Colab Only

In [2]:
from xgboost import XGBClassifier

In [ ]:
xgb_params = {'eta': 0.3,
              'max_depth': 5,
              'subsample': 0.8,
              'colsample_bytree': 0.8,
              'objective': 'binary:logistic',
              'eval_metric': 'auc',
              'seed': 23,
               'tree_method':"gpu_hist"
             }
params = {
    'max_depth': 3,
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 10,
    'n_gpus': 1
}

xg_boost = Pipeline([
    ('classifier', XGBClassifier(**params))
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', xg_boost)
])


In [ ]:
train_predict(pipeline)

AdaBoost


In [36]:
from sklearn.ensemble import AdaBoostClassifier


In [ ]:

adaBoost = Pipeline([
    ('classifier', AdaBoostClassifier()),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', adaBoost)
])


In [ ]:
train_predict(pipeline)


Voting classifier

In [27]:
from sklearn.ensemble import VotingClassifier



In [41]:
models = [
    MultinomialNB(),
    BernoulliNB(),
    LogisticRegression(),
    SGDClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier()
]

m_names = [m.__class__.__name__ for m in models]

models = list(zip(m_names, models))
vc = VotingClassifier(estimators=models)

### Voting Classifier

In [ ]:



votingClassifier = Pipeline([
    ('classifier', vc),
])

pipeline = Pipeline([
    ('vectoriser', tfidf_vectoriser),
    ('classifier', votingClassifier)
])




In [ ]:
train_predict(pipeline)

/home/alex/anaconda3/envs/deepLearning/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
pipeline = make_balance_pipe([TfidfVectorizer(),RandomUnderSampler(),vc])
train_predict(pipeline)



In [10]:

row = df.iloc[[2]]
row['reviewText'].values[0]


'this was a fairly interesting read  it had old style terminology  i was glad to get to read a story that doesnt have coarse  crasslanguage  i read for fun and relaxation  i like the free ebooksbecause i can check out a writer and decide if they are intriguing  innovative  and have enough of the command of englishthat they can convey the story without crude language '